# Image Text Recognition Model Pipeline

The Pipeline has 2 Functions for Output Prediction:
1. **Single Image Prediction:** Takes a single input Image and Gives the Final Text Label as output
2. **Multiple Image Prediction:** Takes list of Image path names of images to be predicted and prints Actual Text Label and its corresponding Predicted Output Text Label for each Image present in the Folder Path

In [1]:
from IPython.display import Image
import numpy as np
import pandas as pd
import os 
import cv2
from datetime import datetime
import keras
import random
import matplotlib.pyplot as plt
from collections import Counter
import itertools
from keras import backend as K
from keras.layers import Input, Conv2D, MaxPool2D, Dense,MaxPooling2D,Bidirectional
from keras.layers import AveragePooling2D, Flatten, Activation
from keras.layers import BatchNormalization, Dropout
from keras.layers import Concatenate, Add, Multiply, Lambda
from keras.layers import UpSampling2D, Reshape
from keras.layers.merge import add,concatenate
from keras.layers import Reshape
from keras.models import Model
from keras.layers.recurrent import LSTM,GRU
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


### 1. Data Loading

**Sampled_data_100.csv** file contains 100 Images sampled from Synth Text Test Data for Pipeline Implementation

In [2]:
test_data=pd.read_csv('Sampled_data_100.csv')

In [3]:
test_data.drop(['Unnamed: 0'],axis=1,inplace=True)
print(test_data.shape)

(100, 2)


In [4]:
test_data.head()

,ImageName,Labels
0,Test_data/501_nonadhesive.jpg,NONADHESIVE
1,Test_data/502_Aglitter.jpg,AGLITTER
2,Test_data/503_azania.jpg,AZANIA
3,Test_data/504_Pliocenes.jpg,PLIOCENES
4,Test_data/505_Cozenage.jpg,COZENAGE


### 2. Utility Functions

In [5]:
#Letters present in the Label Text
letters= '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'

**Integer Labels to Text Label Converter**

Converts a list of integers to corresponding characters and combines them as a string 

In [6]:
def words_from_labels(labels):
    """
    converts the list of encoded integer labels to word strings like eg. [12,10,29] returns CAT 
    """
    txt=[]
    for ele in labels:
        if ele == len(letters): # CTC blank space
            txt.append("")
        else:
            #print(letters[ele])
            txt.append(letters[ele])
    return "".join(txt)

**Best Path Decoding**

The Function Takes the final 48x37 output matrix from the model, and takes the argmax of the matrix across each column (which returns a value between 0 to 36 (ctc_blank) both included). The outputs are then merged for repeated values and gives a list of integers. The Final output integers is then converted to final output string text and it is returned by the function

In [7]:
#https://keras.io/examples/image_ocr/
#https://github.com/qjadud1994/CRNN-Keras
def decode_label(out):
    """
    Takes the predicted ouput matrix from the Model and returns the output text for the image
    """
    # out : (1, 48, 37)
    out_best = list(np.argmax(out[0,2:], axis=1))

    out_best = [k for k, g in itertools.groupby(out_best)]  # remove overlap value

    outstr=words_from_labels(out_best)
    return outstr

### 3. Model Parameters

In [8]:
#image height
img_h=32
#image width
img_w=170
#image Channels
img_c=1
# classes for softmax with number of letters +1 for blank space in ctc
num_classes=len(letters)+1
batch_size=64
max_length=15 # considering max length of ground truths labels to be 15

### 4. Model Architecture

In [9]:
def model_create(drop_out_rate=0.35):
    """
    Builds the model which takes input as images which is used for prediction and returns the Output Matrix 
    of of dimesnions 48x37 where 48 is the number of time-steps of RNN and 37 is the length of letters 
    + 1 character for ctc blank
    """
    
    if K.image_data_format() == 'channels_first':
        input_shape = (1, img_w, img_h)
    else:
        input_shape = (img_w, img_h, 1)
       
    model_input=Input(shape=input_shape,name='img_input',dtype='float32')

    # Convolution layer 
    model = Conv2D(64, (3, 3), padding='same', name='conv1', kernel_initializer='he_normal')(model_input) 
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = MaxPooling2D(pool_size=(2, 2), name='max1')(model) 

    model = Conv2D(128, (3, 3), padding='same', name='conv2', kernel_initializer='he_normal')(model) 
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = MaxPooling2D(pool_size=(2, 2), name='max2')(model) 

    model = Conv2D(256, (3, 3), padding='same', name='conv3', kernel_initializer='he_normal')(model) 
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Conv2D(256, (3, 3), padding='same', name='conv4', kernel_initializer='he_normal')(model)
    model=Dropout(drop_out_rate)(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = MaxPooling2D(pool_size=(1, 2), name='max3')(model)  

    model = Conv2D(512, (3, 3), padding='same', name='conv5', kernel_initializer='he_normal')(model) 
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Conv2D(512, (3, 3), padding='same', name='conv6')(model)
    model=Dropout(drop_out_rate)(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = MaxPooling2D(pool_size=(1, 2), name='max4')(model) 

    model = Conv2D(512, (2, 2), padding='same', kernel_initializer='he_normal', name='con7')(model)
    model=Dropout(0.25)(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)    

    # CNN to RNN
    model = Reshape(target_shape=((42, 1024)), name='reshape')(model)  
    model = Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1')(model)  

    # RNN layer
    model=Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='he_normal'), merge_mode='sum')(model)
    model=Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='he_normal'), merge_mode='concat')(model)

    # transforms RNN output to character activations:
    model = Dense(num_classes, kernel_initializer='he_normal',name='dense2')(model) 
    y_pred = Activation('softmax', name='softmax')(model)

    return Model(inputs=[model_input], outputs=y_pred)

### 5. Image Prediction Pipeline

In [10]:
def test_data_single_image_Prediction(model,test_img_path):
    """
    Takes the best model, test data image paths, test data groud truth labels and pre-processes the input image to 
    appropriate format for the model prediction, takes the predicted output matrix and uses best path decoding to 
    generate predicted text and prints the Predicted Text Label, Time Taken for Computation
    """
    start=datetime.now()
    
    test_img=cv2.imread(test_img_path)
    test_img_resized=cv2.resize(test_img,(170,32))
    test_image=test_img_resized[:,:,1]
    test_image=test_image.T 
    test_image=np.expand_dims(test_image,axis=-1)
    test_image=np.expand_dims(test_image, axis=0)
    test_image=test_image/255
    model_output=model.predict(test_image)
    predicted_output=decode_label(model_output)
    print("Predicted Text in the Image: ", predicted_output)
    print("Time Taken for Processing: ",datetime.now()-start)    

In [11]:
def test_data_multiple_image_Prediction(model,test_img_names,test_labels,total):
    """
    Takes the best model, test data image paths, test data groud truth labels and pre-processes the input image to 
    appropriate format for the model prediction, takes the predicted output matrix and uses best path decoding to 
    generate predicted text and compares with ground truth text for the input and ouputs the final accuracy,
    letter accuracy and  letter count across the entire test set of images
    """
    start=datetime.now()
    accuracy=0
    letter_acc=0
    letter_cnt=0
    count=0
    for i in range(len(test_labels)):
        test_img=cv2.imread(test_img_names[i])
        test_img_resized=cv2.resize(test_img,(170,32))
        test_image=test_img_resized[:,:,1]
        test_image=test_image.T
        test_image=np.expand_dims(test_image,axis=-1)
        test_image=np.expand_dims(test_image, axis=0)
        test_image=test_image/255
        model_output=model.predict(test_image)
        predicted_output=decode_label(model_output)
        actual_output=test_labels[i]
        count+=1
        for j in range(min(len(predicted_output),len(actual_output))):
            if predicted_output[j]==actual_output[j]:
                letter_acc+=1
        letter_cnt+=max(len(predicted_output),len(actual_output))
        if actual_output==predicted_output:
            accuracy+=1
        print("-"*80)
        print("Actual Text: ",actual_output,"   Predicted Text: ",predicted_output)
    print("="*80)
    print("Model Output Accuracy: ",(accuracy/total)*100, " %")
    print("Model Output Letter Accuracy: ",(letter_acc/letter_cnt)*100, " %")
    print("Time Taken for Processing: ",datetime.now()-start)

### 6. Loading Best Model 

In [12]:
model=model_create()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img_input (InputLayer)       (None, 170, 32, 1)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 170, 32, 64)       640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 170, 32, 64)       256       
_________________________________________________________________
activation_1 (Activation)    (None, 170, 32, 64)       0         
_________________________________________________________________
max1 (MaxPooling2D)          (None, 85, 16, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 85, 16, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 85, 16, 128)       512       
__________

The Best Weights for the Model are stored in BestLSTMModelWeights Folder, loading the stored best weights for Model 1 for prediction

In [15]:
model.load_weights('Final_LSTM_Model_Best_Weights/Best_Img_recog_LSTM_Adam_model_run_weights.h5')

### 7. Single Image Prediction

In [16]:
test_image_1='Test_Data/3_Chronographs.jpg'

In [17]:
Image(test_image_1)

In [18]:
test_data_single_image_Prediction(model,test_image_1)

Predicted Text in the Image:  CHRONOGRAPHS
Time Taken for Processing:  0:00:03.264209


In [19]:
test_image_2='Test_Data/36_goldmine.jpg'

In [20]:
Image(test_image_2)

In [21]:
test_data_single_image_Prediction(model,test_image_2)

Predicted Text in the Image:  GOLDMINE
Time Taken for Processing:  0:00:00.083438


In [22]:
test_image_3='Test_Data/48_SCHOOLBOY.jpg'

In [23]:
Image(test_image_3)

In [24]:
test_data_single_image_Prediction(model,test_image_3)

Predicted Text in the Image:  SCHOOLBOY
Time Taken for Processing:  0:00:00.066124


### 8. Multiple Image Prediction

In [25]:
test_img_paths=test_data['ImageName'].values
test_img_labels=test_data['Labels'].values
total=100

In [26]:
test_data_multiple_image_Prediction(model,test_img_paths,test_img_labels,total)

--------------------------------------------------------------------------------
Actual Text:  NONADHESIVE    Predicted Text:  NONADHESIVE
--------------------------------------------------------------------------------
Actual Text:  AGLITTER    Predicted Text:  AGLITTER
--------------------------------------------------------------------------------
Actual Text:  AZANIA    Predicted Text:  AZANIA
--------------------------------------------------------------------------------
Actual Text:  PLIOCENES    Predicted Text:  PLIOCENES
--------------------------------------------------------------------------------
Actual Text:  COZENAGE    Predicted Text:  COZENOGE
--------------------------------------------------------------------------------
Actual Text:  LIEFER    Predicted Text:  LIEFER
--------------------------------------------------------------------------------
Actual Text:  OUTBOXES    Predicted Text:  OUTBOXES
---------------------------------------------------------------------

--------------------------------------------------------------------------------
Actual Text:  INDISSOLUBLE    Predicted Text:  INDISSOLUBLE
--------------------------------------------------------------------------------
Actual Text:  WIST    Predicted Text:  WIST
--------------------------------------------------------------------------------
Actual Text:  BACKBITER    Predicted Text:  BACKBITER
--------------------------------------------------------------------------------
Actual Text:  FITMENT    Predicted Text:  FITMENT
--------------------------------------------------------------------------------
Actual Text:  TRAMMEL    Predicted Text:  TRAMMEL
--------------------------------------------------------------------------------
Actual Text:  RADIATE    Predicted Text:  RADIATE
--------------------------------------------------------------------------------
Actual Text:  SKEPTICAL    Predicted Text:  SKEPTICAL
-----------------------------------------------------------------------